# CIFAR10-CNN-Model Using Pytorch
( Accuracy 67% 이상)  softmax 추가, layer추가 , epoch 더 추가해보기

In [1]:
import torch
import torch.nn.init
import torchvision
from torch.autograd import Variable
import torchvision.utils as utils
import torchvision.datasets as dsets
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5,0.5,0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                       download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                         shuffle=True, num_workers=10)

testset  = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                        shuffle=False, num_workers=10)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [2]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 24, 5)
        self.b1 = nn.BatchNorm2d(24)  #BatchNorm 추가!!
        self.pool = nn.MaxPool2d(2,2)
        
        self.conv2 = nn.Conv2d(24, 64, 5)
        self.b2 = nn.BatchNorm2d(64)
        
        self.fc1 = nn.Linear(64 * 5 * 5, 240)
        self.fc2 = nn.Linear(240, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.b1((self.conv1(x)))))
        x = self.pool(F.relu(self.b2(self.conv2(x))))
        x = x.view(-1, 64 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()
net

Net(
  (conv1): Conv2d(3, 24, kernel_size=(5, 5), stride=(1, 1))
  (b1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(24, 64, kernel_size=(5, 5), stride=(1, 1))
  (b2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=1600, out_features=240, bias=True)
  (fc2): Linear(in_features=240, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [3]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters()) # Adam Optimizer사용!!

In [4]:
for epoch in range(20):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        
        inputs, labels = data
        
        inputs, labels = Variable(inputs), Variable(labels)
        
        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 128 == 127:   # print every 2000 mini-batches
            print('[%d, %5d] loss : %.3f' % (epoch + 1, i+ 1, running_loss / 128))
            running_loss = 0.0
    
    correct = 0
    total = 0
    for data in testloader:
        images, labels = data
        outputs = net(Variable(images))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
        
    print('Accuracy 1000 test images : %d %%'% (100 * correct/total))
    
print('Finished Training')

#torch.save(net.state_dict(), 'CNN.pkl')


[1,   128] loss : 1.610
[1,   256] loss : 1.301
[1,   384] loss : 1.185
Accuracy 1000 test images : 52 %
[2,   128] loss : 1.060
[2,   256] loss : 1.010
[2,   384] loss : 0.986
Accuracy 1000 test images : 57 %
[3,   128] loss : 0.909
[3,   256] loss : 0.867
[3,   384] loss : 0.838
Accuracy 1000 test images : 62 %
[4,   128] loss : 0.773
[4,   256] loss : 0.784
[4,   384] loss : 0.768
Accuracy 1000 test images : 63 %
[5,   128] loss : 0.693
[5,   256] loss : 0.697
[5,   384] loss : 0.697
Accuracy 1000 test images : 65 %
[6,   128] loss : 0.610
[6,   256] loss : 0.635
[6,   384] loss : 0.611
Accuracy 1000 test images : 66 %
[7,   128] loss : 0.530
[7,   256] loss : 0.554
[7,   384] loss : 0.583
Accuracy 1000 test images : 65 %
[8,   128] loss : 0.475
[8,   256] loss : 0.500
[8,   384] loss : 0.518
Accuracy 1000 test images : 65 %
[9,   128] loss : 0.429
[9,   256] loss : 0.441
[9,   384] loss : 0.460
Accuracy 1000 test images : 65 %
[10,   128] loss : 0.377
[10,   256] loss : 0.395
[10, 